In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Importing Libaries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

Loading the dataset

In [ ]:
df = pd.read_csv('../input/hr-comma-sepcsv/HR_comma_sep.csv')

In [ ]:
columns_names=df.columns.tolist()
print("Columns names:")
print(columns_names)

Principal Component Analysis****

In [ ]:
df.head()

In [ ]:
df_drop=df.drop(labels=['sales','salary'],axis=1)
df_drop.head()

df.drop() is the method to drop the columns in our dataframe

In [ ]:
cols = df_drop.columns.tolist()
cols

Here we are converting columns of the data frame to list so it would be easier for us to reshuffle the columns.We are going to use column insert method

In [ ]:
cols.insert(0, cols.pop(cols.index('left')))
cols

**Converting list to columns again**

In [ ]:
df_drop = df_drop.reindex(columns=cols)

In [ ]:
X = df_drop.iloc[:,1:8].values
y = df_drop.iloc[:,0].values
X


In [ ]:
y

In [ ]:
np.shape(X)

Thus X is now matrix with 14999 rows and 7 columns

 ** Data Standardisation **

In [ ]:
from sklearn.preprocessing import StandardScaler
X_std = StandardScaler().fit_transform(X)

** Computing Eigenvectors and Eigenvalues **
First we need to calculate covariance matrix.
   


** Covariance matrix **

In [ ]:
mean_vec = np.mean(X_std, axis=0)
cov_mat = (X_std - mean_vec).T.dot((X_std - mean_vec)) / (X_std.shape[0]-1)
print('Covariance matrix \n%s' %cov_mat)

In [ ]:
print('NumPy covariance matrix:\n%s' %np.cov(X_std.T))

In [ ]:
plt.figure(figsize=(8,8))
sns.heatmap(cov_mat, vmax=1, square=True,annot=True,cmap='cubehelix')
plt.title('Correlation between different features')

** Eigen Decomposition of the covariance matrix **

In [ ]:
eig_vals, eig_vecs = np.linalg.eig(cov_mat)
print('Eigenvectors \n%s' %eig_vecs)
print('\nEigenvalues \n%s' %eig_vals)

** Selecting Principal Components**

In [ ]:
# Make a list of(eigenvalue, eigenvector) tuples
eig_pairs = [(np.abs(eig_vals[i]),eig_vecs[:,i]) for i in range(len(eig_vals))]

# sort the (eigenvalue, eigenvector) tuples from high to low
eig_pairs.sort(key=lambda x:x[0],reverse=True)

# Visually confirm that the list is correctly sorted by decreasing eigenvalues

print('Eigenvalues in decreasing order:')
for i in eig_pairs:
    print(i[0])


In [ ]:
tot = sum(eig_vals)
var_exp = [(i /tot)*100 for i in sorted(eig_vals, reverse=True)]

In [ ]:
with plt.style.context('dark_background'):
    plt.figure(figsize=(6, 4))
    
    plt.bar(range(7), var_exp, alpha=0.5, align='center',
            label='individual explained variance')
    plt.ylabel('Explained variance ratio')
    plt.xlabel('Principal components')
    plt.legend(loc='best')
    plt.tight_layouut()

** Projection Matrix**

In [ ]:
matrix_w = np.hstack((eig_pairs[0][1].reshape(7,1),
                     eig_pairs[1][1].reshape(7,1)
                     ))
print('Matrix W:\n', matrix_w)

In [ ]:
Y=X_std.dot(matrix_w)
Y

** PCA in scikit-learn**

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(X_std)
plt.plot(np.cumsum(pca.explained_variance_ratio_))
plt.xlim(0,7,1)
plt.xlabel('Number of components')
plt.ylabel('Cumulative explained variance')

The above graph shows almost 90% variace by the first 6 components .Therefore we can drop 7th component

In [ ]:
from sklearn.decomposition import PCA
sklearn_pca = PCA(n_components=6)
Y_sklearn = sklearn_pca.fit_transform(X_std)
print(X_std)

In [ ]:
Y_sklearn.shape

Thus principal component Analysis is used to remove the redundant features from the datasets withouut losing much information.These features are less dimensional in nature.The first component has the highest variance followed by second,third and so on.PCA works better on dataset havingthree or higher dimensions. 